In [1]:
#import required libraries
import twitter
import pandas as pd
import string
from nltk.corpus import stopwords
# load previously created naive bayes model 
from sklearn.externals import joblib
nb_model = joblib.load('NB_Model_twitter.pkl')

# Get tokens & secrets from twitter dev account
CONSUMER_KEY = 'o9MERpjWG7EFhE2VjTzXaLZ64'
CONSUMER_SECRET = 'Vh9MS9nMRIxJWAa9q4XCgKTeRVusurzAFQos9mmyMXj2yB3ZJU'
ACCESS_TOKEN_KEY = '3130029268-quvFqyaptfJFHz9mJxWZl9MEyyFlrQOiW37Z5YP'
ACCESS_TOKEN_SECRET = 'VhEOwXzdXrf3kmC3Q79flAaX1NHUm4Md1hvtnIzJlrWuN'

#Create empty dataframes to be populated with trends and tweets of those trends
df_trend_hist = pd.DataFrame(columns=["name","query","timestamp","tweet_volume","url"])
df_trend_current = pd.DataFrame(columns=["name","query","timestamp","tweet_volume","url"])
df_trend_search = pd.DataFrame(columns = ["Hashtag","ID","Created","Text","Clean Text","Place","Coordinates","Favourite Count""Retweet Count","Sentiment"])

#instantiate an api instance
api = twitter.Api(consumer_key = CONSUMER_KEY,
                  consumer_secret = CONSUMER_SECRET,
                  access_token_key= ACCESS_TOKEN_KEY,
                  access_token_secret=ACCESS_TOKEN_SECRET)

def get_trends(Woeid):
    """Uses trends/place endpoint to retrieve current trending Tweets in specified Woeid (where on earth id)
    https://developer.twitter.com/en/docs/trends/trends-for-location/api-reference/get-trends-place
    Populates data frame for current trends and concatenates current trends to historical trends data frame"""
    global df_trend_hist
    global df_trend_current
    #call twitter for trending in geographical location
    trend_response = api.GetTrendsWoeid(Woeid)
    a=[]
    #loop through trend response and update attribute values in a dictionary, append dictionaries to list
    for x in trend_response:
        dictionary = {"name":'',"query":'',"timestamp":'',"tweet_volume":0,"url":''}
        dictionary["name"]=x.name
        dictionary["query"]=x.query
        dictionary["timestamp"]=x.timestamp
        dictionary["tweet_volume"]=x.tweet_volume
        dictionary["url"]=x.url
        a.append(dictionary)
    #create data frame from list of dictionaries
    df_a = pd.DataFrame(a)
    #concatenate historical trend data frame with existing trend dataframe
    df_trend_hist = pd.concat([df_trend_hist,df_a],axis=0,sort=False)
    #over write current trend data frame with new trends
    df_trend_current = df_a

def text_process(message):
    #1 Split message into list of words & remove Stop Words, hashtags, @mentions & http links
    no_punc = [word for word in message.split() if word.lower() not in stopwords.words('english') 
               and word[0] != '#' 
               and word[0] != '@'
               and word[0:4] != 'http']
    #2 rejoin split word list in space separated string
    no_punc = ' '.join(no_punc)
    #3 Parse string for punctuation marks
    no_punc = [char for char in no_punc if char not in string.punctuation]
    #4 Rejoin characters into text string
    no_punc = ''.join(no_punc)
    return no_punc
    
def search_trends():
    """Uses search/tweets endpoint to retrieve Tweets relating to trending tweets
    https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets
    Utilises NB model to classify sentiment of tweet
    concatenates returned tweets with historical tweets data frame
    """
    global df_trend_search
    a = []
    #loop through trend data frame query and name columns
    for query,hashtag in zip(df_trend_current['query'],df_trend_current['name']):
        #check trend search data frame, if entries exist include since_id parameter to prevent duplicates
        if df_trend_search.empty:
            query_string = 'q='+query+'&count=5&lang=en'
        else:
            since_id = df_trend_search['ID'].max()
            query_string = 'q='+query+'&count=5&lang=en&since_id'+str(since_id)
        #perform API search call on twitter
        response = api.GetSearch(raw_query=query_string)
        #loop through response extracting attributes saving to dictionary, append dictionaries to list
        for x in response:
            dictionary = {"Hashtag":'',"ID":'',"Created":'',"Text":'',"Place":'',"Coordinates":'',"Favourite Count":0,"Retweet Count":0}
            dictionary["Hashtag"] = hashtag
            dictionary["ID"] = x.id
            dictionary["Created"] = x.created_at
            dictionary["Text"] = x.text
            dictionary["Place"] = x.place
            dictionary["Coordinates"] = x.coordinates
            dictionary["Favourite Count"] = x.favorite_count
            dictionary["Retweet Count"] = x.retweet_count
            a.append(dictionary)
    #create dataframe from dictionaries list
    df_a = pd.DataFrame(a)
    df_a.head()
    #clean df_a text column using text process function, create clean text column
    df_a['Clean Text'] = df_a['Text'].apply(text_process)
    #create sentiment identifier column in df_a
    df_a['Sentiment'] = nb_model.predict(df_a['Clean Text'])
    #concatenate df_a data frame to df_trend_search
    df_trend_search = pd.concat([df_trend_search,df_a],axis=0,sort=False)
    
# import schedule library to facilitate process automation
import schedule
import time
from datetime import datetime

initial_flag = False

def initial_job():
    global initial_flag
    print('Running initial job, calls Trends and then searches Trends for tweets')
    get_trends(4118)
    search_trends()
    initial_flag = True
    print('Initial flag:'+str(initial_flag))
    return schedule.CancelJob
    
def trend_job():
    if initial_flag == True:
        print('Twitter Trend GET Call')
        print(str(datetime.now()))
        get_trends(4118)
    else:
        Print('Initial flag is'+str(initial_flag)+' trend_job() will not run')

def trend_search_job():
    if initial_flag == True:
        print('Twitter Search Trend GET Call')
        print(str(datetime.now()))
        search_trends()
    else:
        Print('Initial flag is'+str(initial_flag)+' trend_search_job() will not run')


schedule.every(1).seconds.do(initial_job)
schedule.every(15).minutes.do(trend_job)
schedule.every(1).minutes.do(trend_search_job)



Every 1 minute do trend_search_job() (last run: [never], next run: 2018-09-04 00:01:38)

In [2]:
while 1:
    schedule.run_pending()
    time.sleep(1)

Running initial job, calls Trends and then searches Trends for tweets
Initial flag:True


In [3]:
df_trend_current.head()

,name,query,timestamp,tweet_volume,url
0,Nike,Nike,2018-09-04T04:00:43Z,1294582.0,http://twitter.com/search?q=Nike
1,#Ticats,%23Ticats,2018-09-04T04:00:43Z,NaN,http://twitter.com/search?q=%23Ticats
2,#BachelorInParadise,%23BachelorInParadise,2018-09-04T04:00:43Z,35912.0,http://twitter.com/search?q=%23BachelorInParadise
3,Marcus Stroman,%22Marcus+Stroman%22,2018-09-04T04:00:43Z,NaN,http://twitter.com/search?q=%22Marcus+Stroman%22
4,New Yorker,%22New+Yorker%22,2018-09-04T04:00:43Z,124059.0,http://twitter.com/search?q=%22New+Yorker%22


In [4]:
df_trend_hist.head()

,name,query,timestamp,tweet_volume,url
0,Nike,Nike,2018-09-04T04:00:43Z,1294582.0,http://twitter.com/search?q=Nike
1,#Ticats,%23Ticats,2018-09-04T04:00:43Z,NaN,http://twitter.com/search?q=%23Ticats
2,#BachelorInParadise,%23BachelorInParadise,2018-09-04T04:00:43Z,35912.0,http://twitter.com/search?q=%23BachelorInParadise
3,Marcus Stroman,%22Marcus+Stroman%22,2018-09-04T04:00:43Z,NaN,http://twitter.com/search?q=%22Marcus+Stroman%22
4,New Yorker,%22New+Yorker%22,2018-09-04T04:00:43Z,124059.0,http://twitter.com/search?q=%22New+Yorker%22


In [5]:
df_trend_search.head()

,Hashtag,ID,Created,Text,Clean Text,Place,Coordinates,Favourite CountRetweet Count,Sentiment,Favourite Count,Retweet Count
0,Nike,1036826306774794240,Tue Sep 04 04:00:29 +0000 2018,"RT @ManInTheHoody: lol, u can see the socks he...",RT lol u see socks wearing cut dude went found...,None,None,NaN,1.0,0.0,85.0
1,Nike,1036826306766352385,Tue Sep 04 04:00:29 +0000 2018,😂 RT @andylassner: Based on the success of the...,😂 RT Based success right wing boycotts “Hamilt...,None,None,NaN,1.0,0.0,0.0
2,Nike,1036826306741235714,Tue Sep 04 04:00:29 +0000 2018,@johnrich @Nike I own nothing nike because I'm...,nothing nike Im disabled poor lmao 😹 guess ahe...,None,None,NaN,0.0,0.0,0.0
3,Nike,1036826306724343809,Tue Sep 04 04:00:29 +0000 2018,@comradebernout @msmoon61 @johnrich @Nike @Kae...,Im sure said expansion freedom,None,None,NaN,1.0,0.0,0.0
4,Nike,1036826306523082752,Tue Sep 04 04:00:29 +0000 2018,@darcangel21 This country was built on #strong...,country built people standing people don’t vo…,None,None,NaN,1.0,0.0,0.0
